<a href="https://colab.research.google.com/github/jotaperes/GeocodingReverse/blob/main/Geocoding_Reverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Geocoding Reverse

Recentemente, surgiu uma demanda no meu trabalho relacionada a necessidade de descobrir endereços a partir das coordenadas de um determinado objeto. Então, achei que seria um projeto interessante para desenvolver.

Após algumas tentativas e testes diversos, cheguei a um código bem enxuto que realiza de forma bem prática e fácil todo o processo.

Tive como referência as seguintes fontes:

https://geopy.readthedocs.io/en/stable/#

https://towardsdatascience.com/reverse-geocoding-in-python-a915acf29eb6#:~:text=Reverse%20Geocoding%20is%20an%20essential,features%20is%20called%20Reverse%20Geocoding.

In [1]:
!pip install tqdm

import geopandas as gpds #https://geopandas.org/en/stable/index.html
import pandas as pd
import geopy 
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter #Necessário para grandes quantidades de acessos, o que não é o caso presente, mas vale deixar por aqui.
import tqdm #biblioteca para visualizar barra de carregamento
from tqdm._tqdm_notebook import tqdm_notebook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  if __name__ == '__main__':


In [3]:
#Testando o reverse e analisando como os dados são extraídos.

coordinates = '-22.8810148, -43.0967501'
location = locator.reverse(coordinates, language = 'pt-br')
location.raw

{'address': {'ISO3166-2-lvl4': 'BR-RJ',
  'amenity': 'Posto Legal',
  'city': 'Niterói',
  'city_district': 'Região Norte',
  'country': 'Brasil',
  'country_code': 'br',
  'county': 'Região Metropolitana do Rio de Janeiro',
  'municipality': 'Região Geográfica Imediata do Rio de Janeiro',
  'postcode': '24130-082',
  'region': 'Região Sudeste',
  'road': 'Alameda São Boaventura',
  'state': 'Rio de Janeiro',
  'state_district': 'Região Geográfica Intermediária do Rio de Janeiro',
  'suburb': 'Fonseca'},
 'boundingbox': ['-22.881218', '-22.881118', '-43.0967059', '-43.0966059'],
 'display_name': 'Posto Legal, Alameda São Boaventura, Fonseca, Região Norte, Niterói, Região Geográfica Imediata do Rio de Janeiro, Região Metropolitana do Rio de Janeiro, Região Geográfica Intermediária do Rio de Janeiro, Rio de Janeiro, Região Sudeste, 24130-082, Brasil',
 'lat': '-22.881168',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'lon': '-43.0966559',
 'osm_i

Testando o geopandas com o endereço do Barra Shopping presente no Google.
Cabe esclarecer que ele dá o endereço aproximado e que as bases são diferentes da API do Google.
Aparentemente está tudo certo.

In [20]:
local = 'Av. das Américas, 4666 - Barra da Tijuca, Rio de Janeiro - RJ, 22640-102'
gpds.tools.geocode(local, provider = 'nominatim', user_agent = "assinaturasjoaopedro@gmail.com", country_bias = 'Brazil')

,geometry,address
0,POINT (-43.35981 -22.99726),"Centro Médico BarraShopping, 4666, Avenida das..."


In [4]:
#importando os dados
df = pd.read_excel('localizacoes.xlsx')
df.head()

,ID,Local,Y,X
0,1,Shopping Plaza Niterói,-22.896462,-43.126116
1,2,Shopping Campos,-21.762892,-41.346851
2,3,Barra Shopping,-22.976356,-43.361848


O geopy apenas aceita os dados no formato f'{y},{x}'. Por isso, é preciso criar essa nova coluna que chamei de geom.


In [5]:
df['geom'] = df["Y"].map(str) + ','+ df['X'].map(str)
df['geom'][0]

'-22.8964615,-43.1261158'

In [6]:
locator = Nominatim(user_agent='assinaturasjoaopedro@gmail.com', timeout=10) #definindo o user_agent novamente
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001) #definindo o ratelimiter

Aqui, eu criei um loop que basicamente faz a requisição da informação para cada linha de meu banco de dados e salva primeiro em algumas variáveis, que depois são concatenadas em uma lista. Essa lista posteriormente se transforma em um dataframe.

In [7]:
endereco_completo=[]

for i in tqdm(range(len(df))):
    location = locator.reverse(df['geom'][i], language = 'pt-br')
    object_id = df['ID'][i]
    cidade = location.raw['address']['city']
    estado = location.raw['address']['state']
    geom = location.raw['lat'] + ',' + location.raw['lon']
    endereco_completo.append([cidade, estado, geom, location, object_id])


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
df_endereco = pd.DataFrame(endereco_completo, columns=["pais", "estado", "geom", "location", "id"])

df_endereco

,pais,estado,geom,location,id
0,Niterói,Rio de Janeiro,"-22.8966197,-43.1264197","(Só Pensa no Quilo, Avenida Visconde do Rio Br...",1
1,Campos dos Goytacazes,Rio de Janeiro,"-21.76299619382721,-41.34713920037181","(Rodovia Governador Mário Covas, Parque Santo ...",2
2,Rio de Janeiro,Rio de Janeiro,"-22.9773473,-43.3629499","(Avenida Engenheiro Hermano Jordão Freire, Bar...",3


#Enfim, o final

Realizo um merge com os dois bancos.

In [9]:
df = df.merge(df_endereco, left_on='ID', right_on='id')

In [10]:
df.head()

,ID,Local,Y,X,geom_x,pais,estado,geom_y,location,id
0,1,Shopping Plaza Niterói,-22.896462,-43.126116,"-22.8964615,-43.1261158",Niterói,Rio de Janeiro,"-22.8966197,-43.1264197","(Só Pensa no Quilo, Avenida Visconde do Rio Br...",1
1,2,Shopping Campos,-21.762892,-41.346851,"-21.7628923,-41.3468512",Campos dos Goytacazes,Rio de Janeiro,"-21.76299619382721,-41.34713920037181","(Rodovia Governador Mário Covas, Parque Santo ...",2
2,3,Barra Shopping,-22.976356,-43.361848,"-22.9763558,-43.3618478",Rio de Janeiro,Rio de Janeiro,"-22.9773473,-43.3629499","(Avenida Engenheiro Hermano Jordão Freire, Bar...",3
